# Análise exploratória de dados

In [ ]:
### Import de bibliotecas para manipulação dos dados 
import pandas as pd
import numpy as np
import math

### para visualizacao 
import seaborn as sns
import matplotlib as plt

### para obter estatisticas de teste
import scipy.stats as stats
from scipy.stats import norm
from sklearn.preprocessing import LabelEncoder

## 1. Descrição dos dados

##### O dataset "Orange Quality Data.csv" contém 241 linhas e 11 colunas, e ele foi retirado dos datasets da kaggle. 
##### As variáveis incluem: 


* **Size (cm):** Diâmetro da laranja em centímetros, representando o tamanho da fruta.
* **Weight (g):** Peso da laranja em gramas, indicando a massa da fruta.
* **Brix (Sweetness):** Nível de açúcar da laranja, medido pela escala Brix, que indica o grau de doçura.
* **pH (Acidity):** Nível de acidez da laranja, medido pelo pH, que indica o sabor azedo da fruta.
* **Softness (1-5):** Nível de maciez da laranja em uma escala de 1 a 5, onde 1 indica uma fruta muito dura e 5 indica uma fruta muito macia.
* **HarvestTime (days):** Número de dias desde o plantio até a colheita, indicando o tempo de cultivo.
* **Ripeness (1-5):** Nível de maturação da laranja em uma escala de 1 a 5, onde 1 indica uma fruta verde e 5 indica uma fruta madura.
* **Color:** Cor da casca da laranja, indicando a variedade e o estágio de maturação.
* **Variety:** Variedade da laranja, como Valencia, Navel, Cara Cara, etc.
* **Blemishes (Y/N):** Indica se a laranja possui manchas ou defeitos visíveis (Sim ou Não).
* **Quality (1-5):** Qualidade geral da laranja em uma escala de 1 a 5, avaliando diversos atributos como tamanho, aparência, sabor e textura.

In [ ]:
#Caminho e leitura do Dataframe
df = pd.read_csv('../data/raw/Orange_Quality_Data.csv')

#Análise das 10 primeiras linhas 
df.head(10)

In [ ]:
def  separador():
    print("=" * 50 + " ************** " + "=" * 50)
separador()

# Exibindo as informações do DataFrame
print(f"Informações gerais do DataFrame")
print(f" ")
df.info()
print(f" ")

separador()
print(f"Resumo estatístico")
print(f" ")
#Resumo estatístico
df.describe()

In [ ]:
# Caminho do dicionário dos dados .csv
csv_path = '../data/external/dicionario_de_dados_laranjas.csv'

# Leitura do CSV
df_dicionario_dados = pd.read_csv(csv_path, sep=';')
dicionario = df_dicionario_dados.loc[:, ~df_dicionario_dados.columns.str.contains('^Unnamed')]
print(dicionario)

## 2. Perguntas de partida e hipóteses

## a: Como a maciez das laranjas varia com o tempo de colheita e o estágio de maturação? Laranjas mais maduras são sempre mais macias?

In [ ]:
# Análise Exploratória
print(f"Análise descritiva dos dados:")
print(df[['Softness (1-5)', 'HarvestTime (days)', 'Ripeness (1-5)']].describe())
print(' ')

# Verificando a correlação entre as variáveis Softness (1-5), HarvestTime (days) e Ripeness (1-5)
correlation = df[['Softness (1-5)', 'HarvestTime (days)', 'Ripeness (1-5)']].corr()
print("Correlação entre as variáveis Softness (1-5), HarvestTime (days) e Ripeness (1-5):")
print(correlation)

# Gráfico de Dispersão com Matriz de Cores para Estágio de Maturação (Ripeness (1-5))
plt.figure(figsize=(12, 8))
sns.scatterplot(data=df, x='HarvestTime (days)', y='Softness (1-5)', hue='Ripeness (1-5)', palette="plasma", s=100, edgecolor="w", alpha=0.7)

# Configurações do Gráfico
plt.title('Relação entre Tempo de Colheita, Maciez e Maturação das Laranjas')
plt.xlabel('Tempo de Colheita (dias)')
plt.ylabel('Maciez (1-5)')
plt.legend(title='Estágio de Maturação', loc='best')
plt.grid(True, linestyle='--', alpha=0.6)

# Exibir o gráfico
plt.show()

## b: A cor das laranjas está relacionada à sua qualidade? Quais tonalidades são indicativas de maior qualidade?

In [ ]:
# Análise Exploratória da Relação entre Cor e Qualidade
# Calculando a média da qualidade para cada tonalidade de cor
color_quality = df.groupby('Color')['Quality (1-5)'].mean().sort_values()

# Exibindo a tabela de médias de qualidade por cor para melhor visualização 
print("Qualidade média por tonalidade de cor:")
print(color_quality)

# Gráfico de Barra Qualidade vs Tonalidade de Cor
plt.figure(figsize=(12, 8))
sns.barplot(x=color_quality.index, y=color_quality.values, palette="plasma")

# Configurações do Gráfico
plt.title('Qualidade Média das Laranjas por Tonalidade de Cor')
plt.xlabel('Tonalidade de Cor')
plt.ylabel('Qualidade Média')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.6)

# Adicionando os valores em cima das barras
for index, value in enumerate(color_quality.values):
    plt.text(index, value + 0.05, f"{value:.2f}", ha='center', va='bottom', fontsize=10)


# Exibir o gráfico
plt.show()

#Espaçamento para melhor visualização 
print(' ')
print(' ')
print(' ')
print(' ')

# Transformando a coluna 'color' em valores numéricos para utilizar a correlação
label_encoder = LabelEncoder()
df['color_encoded'] = label_encoder.fit_transform(df['Color'])

# Cálculo da correlação entre cor e qualidade
correlation = df[['color_encoded', 'Quality (1-5)']].corr().iloc[0, 1]
print(f"Correlação entre cor e qualidade: {correlation:.2f}")

# Gráfico de dispersão
plt.figure(figsize=(10, 6))
sns.scatterplot(x='color_encoded', y='Quality (1-5)', data=df, palette="plasma", hue='Color', legend=False)

# Configurações do gráfico de dispersão
plt.title('Correlação entre Cor das Laranjas e Qualidade')
plt.xlabel('Cor (Codificada)')
plt.ylabel('Qualidade')
plt.grid(True, linestyle='--', alpha=0.6)

# Exibir o gráfico
plt.show()

# Exibir cores originais, codificadas e qualidade em tabela para melhor entendimento
color_mapping = pd.DataFrame({
    'Original Color': label_encoder.classes_,  # Cores originais
    'Encoded Color': range(len(label_encoder.classes_)),  # Cores codificadas
    'Quality': [df[df['Color'] == color]['Quality (1-5)'].mean() for color in label_encoder.classes_]  # Média da qualidade por cor
})

print("\nTabela de Cores e Qualidade:")
print(color_mapping)

## c: Qual é a influência das manchas na avaliação da qualidade das laranjas?

In [ ]:
# Remover linhas onde "Blemishes (Y/N)" ou "Quality (1-5)" são NaN
df_clean = df.dropna(subset=['Blemishes (Y/N)', 'Quality (1-5)'])

# Padronizando a coluna 'Blemishes (Y/N)' para conter apenas 'Y' ou 'N' devido a outras informações que contém na resposta
df['Blemishes (Y/N)'] = df['Blemishes (Y/N)'].apply(lambda x: 'Y' if 'Y' in str(x) else 'N')

# Exibi valores únicos para confirmar a padronização
print("Valores únicos após padronização:", df['Blemishes (Y/N)'].unique())

# Convertendo 'Blemishes (Y/N)' para valores booleanos
df['Blemishes (Y/N)'] = df['Blemishes (Y/N)'].apply(lambda x: True if x == 'Y' else False)

# Remover linhas onde "Blemishes (Y/N)" ou "Quality (1-5)" são NaN
df_clean = df.dropna(subset=['Blemishes (Y/N)', 'Quality (1-5)'])
# Resumo estatístico da qualidade com e sem manchas
quality_with_blemishes = df_clean[df_clean['Blemishes (Y/N)'] == True]['Quality (1-5)']
quality_without_blemishes = df_clean[df_clean['Blemishes (Y/N)'] == False]['Quality (1-5)']

# Exibindo as médias para comparação com duas casas decimais
print("Qualidade média das laranjas com manchas:", round(quality_with_blemishes.mean(), 2))
print("Qualidade média das laranjas sem manchas:", round(quality_without_blemishes.mean(), 2))

#Visualização
plt.figure(figsize=(10, 6))
sns.stripplot(data=df_clean, x='Blemishes (Y/N)', y='Quality (1-5)', palette=["#FFA07A", "#98FB98"], jitter=True)
plt.title("Distribuição da Qualidade das Laranjas com e sem Manchas")
plt.xlabel("Manchas")
plt.ylabel("Qualidade")
plt.show()


## d: Como o tempo de colheita afeta as características das laranjas? Existem períodos ideais para a colheita que maximizam a qualidade?

In [ ]:

# gráfico de dispersão tamanho vs colheita
plt.figure(figsize=(10,6))
plt.scatter(df['HarvestTime (days)'], df['Size (cm)'], alpha=0.6, color='pink')
plt.title('Impacto do período de colheita no tamanho da laranja')
plt.xlabel('Período de colheita (dias)')
plt.ylabel('Tamanho da laranja (cm)')
plt.grid(True)
plt.show()

#gráfico de dispersão entre colheita e peso
plt.figure(figsize=(10,6))
plt.scatter(df['HarvestTime (days)'], df['Weight (g)'], alpha=0.6, color='orange')
plt.title('Relação entre tempo de colheita e peso da laranja')
plt.xlabel('Tempo de colheita (dias)')
plt.ylabel('Peso da laranja (g)')
plt.grid(True)
plt.show()


# gráfico com a variável 'Brix (Sweetness)' isolada
plt.figure(figsize=(10, 6), facecolor='black')  
ax1 = plt.gca()  
ax1.set_facecolor('black')  

# linha de Brix (Sweetness)
sns.lineplot(data=df, x='HarvestTime (days)', y='Brix (Sweetness)', label='Doçura (5.5-16.0)', color='yellow', linewidth=2, ci=None)

# Adicionando a linha de tendência (regressão linear) em vermelho
sns.regplot(data=df, x='HarvestTime (days)', y='Brix (Sweetness)', scatter=False, color='red', line_kws={'linewidth': 2, 'color': 'red', 'linestyle': '--'})

# Ajuste de título, rótulos e legenda
plt.title('Impacto do tempo de colheita na doçura das laranjas', color='white', fontsize=16)
plt.xlabel('Tempo de colheita (dias)', color='white', fontsize=12)
plt.ylabel('Doçura (brix)', color='white', fontsize=12)
plt.xticks(color='white', rotation=0)
plt.yticks(color='white')
plt.legend(title='Doçura', title_fontsize=12, facecolor='black', edgecolor='white', labelcolor='white')
plt.grid(True, color='gray', linestyle='--', linewidth=0.5)

plt.show()

# gráfico com as outras variáveis (Softness, pH, Quality)
plt.figure(figsize=(10, 6), facecolor='black')  
ax2 = plt.gca()  
ax2.set_facecolor('black')  

sns.lineplot(data=df, x='HarvestTime (days)', y='Softness (1-5)', label='Maciez (1-5)', color='cyan', linewidth=2, ci=None)
sns.lineplot(data=df, x='HarvestTime (days)', y='pH (Acidity)', label='Acidez (pH)', color='magenta', linewidth=2, ci=None)
sns.lineplot(data=df, x='HarvestTime (days)', y='Quality (1-5)', label='Qualidade (1-5)', color='lime', linewidth=2, ci=None)

# Ajuste de título, rótulos e legenda
plt.title('Impacto do tempo de colheita nas características das laranjas', color='white', fontsize=16)
plt.xlabel('Tempo de colheita (dias)', color='white', fontsize=12)
plt.ylabel('Características das laranjas', color='white', fontsize=12)
plt.xticks(color='white', rotation=0)
plt.yticks(color='white')
plt.legend(title='Características', title_fontsize=12, facecolor='black', edgecolor='white', labelcolor='white')

# Configurar grade com cor personalizada
plt.grid(True, color='gray', linestyle='--', linewidth=0.5)

# Exibir gráfico com as outras variáveis
plt.show()



fig4, (ax4, ax5) = plt.subplots(1, 2, figsize=(14, 5))

# Gráfico de dispersão para Doçura (Brix) vs Tempo de Colheita
sns.scatterplot(data=df, x='HarvestTime (days)', y='Brix (Sweetness)', ax=ax4, hue='Brix (Sweetness)', palette="magma")
ax4.set_title("Doçura (Brix) vs Tempo de Colheita")
ax4.set_xlabel("Tempo de Colheita (dias)")
ax4.set_ylabel("Doçura (Brix)")

# Gráfico de dispersão para pH (Acidez) vs Tempo de Colheita
sns.scatterplot(data=df, x='HarvestTime (days)', y='pH (Acidity)', ax=ax5, hue='pH (Acidity)', palette="magma")
ax5.set_title("pH (Acidez) vs Tempo de Colheita")
ax5.set_xlabel("Tempo de Colheita (dias)")
ax5.set_ylabel("pH (Acidez)")

plt.tight_layout(pad=3.0)
plt.show()
'''


## e: A presença de manchas afeta o pH ou a doçura das laranjas? Isso poderia indicar alguma relação entre defeitos visuais e características internas da fruta.

In [ ]:
# Limpeza da coluna 'Blemishes (Y/N)' para conter apenas 'Y' e 'N'
df['Blemishes (Y/N)'] = df['Blemishes (Y/N)'].apply(lambda x: 'Y' if 'Y' in str(x).upper() else 'N')

# Contar as ocorrências de 'Y' e 'N' após a limpeza
contagem_manchas = df['Blemishes (Y/N)'].value_counts()
print("Contagem após limpeza:")
print(contagem_manchas)

# Separar os dados entre laranjas com e sem manchas
laranjas_com_manchas = df[df['Blemishes (Y/N)'] == 'Y']
laranjas_sem_manchas = df[df['Blemishes (Y/N)'] == 'N']

# Calcular as médias de pH e doçura para cada grupo
media_ph_com_manchas = laranjas_com_manchas['pH (Acidity)'].mean()
media_ph_sem_manchas = laranjas_sem_manchas['pH (Acidity)'].mean()
media_brix_com_manchas = laranjas_com_manchas['Brix (Sweetness)'].mean()
media_brix_sem_manchas = laranjas_sem_manchas['Brix (Sweetness)'].mean()

# Teste t para verificar a significância das diferenças de pH
teste_ph = ttest_ind(laranjas_com_manchas['pH (Acidity)'], laranjas_sem_manchas['pH (Acidity)'])
teste_brix = ttest_ind(laranjas_com_manchas['Brix (Sweetness)'], laranjas_sem_manchas['Brix (Sweetness)'])

# Exibir os resultados
print("\nMédia de pH com manchas:", media_ph_com_manchas)
print("Média de pH sem manchas:", media_ph_sem_manchas)
print("Resultado do teste t para pH:", teste_ph)

print("\nMédia de Brix com manchas:", media_brix_com_manchas)
print("Média de Brix sem manchas:", media_brix_sem_manchas)
print("Resultado do teste t para Brix:", teste_brix)

# Conclusão com base no p-valor
if teste_ph.pvalue < 0.05:
    print("\nDiferença significativa no pH entre laranjas com e sem manchas.")
else:
    print("\nNão há diferença significativa no pH entre laranjas com e sem manchas.")

if teste_brix.pvalue < 0.05:
    print("Diferença significativa na doçura entre laranjas com e sem manchas.")
else:
    print("Não há diferença significativa na doçura entre laranjas com e sem manchas.")

    # Dados para o gráfico
grupos = ['Com Manchas', 'Sem Manchas']
ph_means = [media_ph_com_manchas, media_ph_sem_manchas]
brix_means = [media_brix_com_manchas, media_brix_sem_manchas]

# Criar o gráfico de barras
fig, ax = plt.subplots(figsize=(10, 6))
bar_width = 0.35
index = range(len(grupos))

# Plotar as barras de pH e Brix
bar1 = ax.bar(index, ph_means, bar_width, label='pH (Acidity)', color='skyblue')
bar2 = ax.bar([i + bar_width for i in index], brix_means, bar_width, label='Brix (Sweetness)', color='salmon')

# Configurar os rótulos e título
ax.set_xlabel('Presença de Manchas')
ax.set_ylabel('Média')
ax.set_title('Comparação de pH e Doçura entre Laranjas com e sem Manchas')
ax.set_xticks([i + bar_width / 2 for i in index])
ax.set_xticklabels(grupos)
ax.legend()

# Adicionar os valores em cima de cada barra
for bar in bar1 + bar2:
    height = bar.get_height()
    ax.annotate(f'{height:.2f}',
                xy=(bar.get_x() + bar.get_width() / 2, height),
                xytext=(0, 3),  # 3 points de deslocamento
                textcoords="offset points",
                ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 3. Insights

## a: Como a maciez das laranjas varia com o tempo de colheita e o estágio de maturação? Laranjas mais maduras são sempre mais macias?

As laranjas apresentaram uma maciez média de 3.07 em uma escala de 1 a 5, indicando uma maciez moderada. O tempo de colheita varia de 
4 a 25 dias (média de 15 dias) e mostrou uma correlação fraca, mas positiva (0.331) com a maciez, sugerindo que laranjas colhidas mais 
tarde tendem a ser um pouco mais macias, embora essa relação não seja forte. Quanto à maturação, com média de 3.6 (em uma escala de 1 a 5),
houve uma correlação fraca e negativa (-0.202) com a maciez, sugerindo que, ao contrário do esperado, as laranjas mais maduras não são 
necessariamente mais macias — na verdade, a maciez pode diminuir levemente conforme a maturação aumenta. Conclui-se que laranjas mais 
maduras não são sempre mais macias, e que outros fatores, como a variedade e as condições de cultivo, podem influenciar essa característica. 

## b: A cor das laranjas está relacionada à sua qualidade? Quais tonalidades são indicativas de maior qualidade?

A correlação calculada é de -0.12.
Interpretação dos Resultados
Qualidade Média: A análise mostra que a tonalidade Deep Orange apresenta a maior qualidade média (4.31), 
seguida por Yellow-Orange (4.22). Isso indica que as laranjas dessas cores tendem a ser avaliadas como de maior 
qualidade. Tonalidades Menores: As tonalidades Light Orange, Orange, e Orange-Red têm médias de qualidade inferiores 
(3.28 a 3.75), sugerindo que essas cores estão associadas a uma qualidade menor em comparação às tonalidades mais escuras.
Correlação Negativa: A correlação de -0.12 indica uma relação fraca e negativa entre a cor das laranjas e sua qualidade.
Isso sugere que, embora existam variações na qualidade média entre as diferentes tonalidades, não há uma relação forte que
indique que a cor mais intensa sempre resulta em maior qualidade.

## c: Qual é a influência das manchas na avaliação da qualidade das laranjas?

Se as médias de qualidade das laranjas com manchas forem mais baixas, isso pode indicar que as manchas afetam negativamente a percepção de qualidade, talvez devido a uma associação com defeitos estéticos ou defeitos que afetam o sabor e a textura.
Se a distribuição da qualidade das laranjas com manchas for mais dispersa (com mais variação), pode sugerir que enquanto algumas laranjas com manchas podem ainda ter boa qualidade, outras têm qualidade significativamente inferior.
Se a média de qualidade das laranjas sem manchas for mais alta, podemos concluir que as manchas provavelmente indicam algum defeito que compromete a qualidade geral do produto.
Essas análises podem ser complementadas por métodos estatísticos, como o teste t para comparar as médias e verificar se a diferença é estatisticamente significativa.


## Doçura (Brix) vs. Tempo de Colheita:
* O gráfico mostra que, conforme o tempo de colheita aumenta, a doçura das laranjas, medida pelo índice Brix, também tende a aumentar. 
Isso significa que, quanto mais tempo a laranja permanece na árvore, mais doce ela fica, provavelmente por causa do aumento de açúcar à medida que o fruto amadurece.

## pH (Acidez) vs. Tempo de Colheita:
* O gráfico do pH indica que a acidez das laranjas diminui conforme o tempo de colheita aumenta. 
Ou seja, quanto mais tempo a laranja fica no pé, menos ácida ela se torna. 
O aumento do pH sugere que o fruto fica mais suave ao paladar à medida que amadurece.

Essencialmente, laranjas colhidas mais tarde tendem a ser mais doces e menos ácidas, o que é um benefício na busca por frutas de melhor qualidade.


## e: A presença de manchas afeta o pH ou a doçura das laranjas? Isso poderia indicar alguma relação entre defeitos visuais e características internas da fruta.


Laranjas com manchas têm um pH médio de 3,63, significativamente maior que o pH de 3,38 das laranjas sem manchas, 
indicando uma leve diferença de acidez entre elas. Em relação à doçura, medida pelo índice Brix, não houve diferença 
significativa, com médias de 10,72 para laranjas com manchas e 11,02 para as sem manchas. Assim, enquanto as manchas afetam 
o pH, elas não influenciam a doçura da fruta.